# 0. Import & Load the Raw Data

In [2]:
import numpy as np
import pandas as pd
import os

import time
from datetime import timedelta
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load the raw log data
raw = pd.read_csv(os.path.join(os.getcwd(),'log.csv'))

In [3]:
raw.shape

(5948704, 7)

# 1. Basic Preprocess

## 1.1. Merge the Label Data

In [5]:
# Load the pre-defined label list
labels = pd.read_csv(os.path.join(os.getcwd(),'label.csv'))

In [7]:
# Convert the data type of "datetime" column
# Extract integral parts from the "page" column and save it as the "PAGE" column (in order to match the column name with the pre-defined label.csv file)
raw['datetime'] = pd.to_datetime(raw['datetime'])
raw['PAGE'] = raw['page'].apply(lambda x: str(x.split('/')[4:]))

In [8]:
# Merge the pre-defined label data and the raw log.csv
raw = raw.merge(labels,on='PAGE',how='left')

## 1.2. Converting the data types of columns "datetime", "page"

In [9]:
# Convert the data type of "datetime" column (once again)
# Drop the useless columns ("page", "Num")
raw['datetime'] = pd.to_datetime(raw['datetime'])
raw.drop(['page','Num','URL'],axis=1,inplace=True)

In [10]:
# Change the name of column "PAGE" into "page" (for convenience)
# Sort the columns "user_id" and "datetime" by an ascending order
raw.rename(columns={'PAGE':'page'},inplace=True)
raw.sort_values(['user_id','datetime'],inplace=True)

## 1.3. Remove the outlier logs & Split the sessions

In [11]:
# Remove the logs labels as "*"
raw = raw[raw['Labeling']!='*'].reset_index().drop('index',axis=1)

In [12]:
# Since only numeric labels are left, convert the data type of "Labeling" column into int
raw['Labeling'] = raw['Labeling'].astype(int)

In [13]:
# Divide the session if the difference between two adjacent logs are larger than 60 minutes
# Create a new "id" column by combining the "user_id" column and the session number
raw['session'] = raw.groupby('user_id')['datetime'].apply(lambda x: x.diff().gt('60Min').cumsum())
raw['id'] = raw['user_id']+'_'+raw['session'].astype(str)

In [14]:
# Create the column "page_stay_time(how long did a user stayed in the page)"
raw['page_stay_time'] = raw.groupby('id')['datetime'].diff()

In [15]:
# Remove the logs that have the duplicated URL with the previous log
raw['prev_url'] = raw.groupby('id')['url'].shift(1)
raw = raw[raw['url']!=raw['prev_url']].drop('prev_url',axis=1)

In [16]:
raw.shape

(5394352, 11)

## 1.4. Create basic featueres for each session

In [17]:
# Create the "session_length" column
# Create the "session_nth(nth log ion the session)" column
raw = raw.merge(pd.DataFrame(raw.groupby('id')['session'].count()).rename(columns={'session':'session_length'}),on='id',how='left')
raw['session_nth'] = raw.groupby('id').cumcount()+1

In [18]:
# Create the "prev_Labeling(previous label within the session)" column
# Create the "prev_page(previous page within the session)" column
raw['prev_Labeling'] = raw.groupby('id')['Labeling'].shift(1)
raw['prev_page'] = raw.groupby('id')['page'].shift(1)

In [19]:
# Create the "begin_time(beginning timestamp of the session)" column
# Create the "end_time(ending timestamp of the session)" column
raw = raw.merge(pd.DataFrame(raw.groupby('id')['datetime'].min()).rename(columns={'datetime':'begin_time'}),on='id',how='left')
raw = raw.merge(pd.DataFrame(raw.groupby('id')['datetime'].max()).rename(columns={'datetime':'end_time'}),on='id',how='left')

# 2. Merge the Log&Label data with Cart, Like, Purchase data

In [ ]:
# Load the raw Cart, Like, Purchase data
cart = pd.read_csv(os.path.join(os.getcwd(),'cart.csv'))
like = pd.read_csv(os.path.join(os.getcwd(),'like.csv'))
purchase = pd.read_csv(os.path.join(os.getcwd(),'purchase.csv'))

In [21]:
cart.shape

(212770, 8)

In [22]:
like.shape

(92147, 9)

In [23]:
purchase.shape

(123985, 11)

In [24]:
# For the merging task, create the "end_time2(60 minutes added to the ending timestamp of the session)" column
raw['end_time2'] = raw['end_time'] + pd.to_timedelta(1, unit='h')

## 2.1. Merge with the Cart data

> If there exists an action of the particular user in Cart.csv, check if any of that user's session (defined by the log data) contains the action's timestamp



> If such session session exists, match the action in Cart.csv to the nearest (in time) log in that session

In [ ]:
# Sort the Cart data by user_id and timestamp
# For the merging task, round the timestamp in Cart data up to a second
# Since Cart data is recorded based on UTC+9(hour) system, substract 9 hours in order to match the timestamp of Raw data
# Extract integral part from the "event_type" column of the Cart data

cart.sort_values(by=['user_id','event_ymdt'],inplace=True)
cart['event_ymdt'] = pd.to_datetime(cart['event_ymdt']).round('s')
cart['event_ymdt'] = cart['event_ymdt'] - timedelta(hours=9)   
cart['event_type'] = cart['event_type'].apply(lambda x: x.split('.')[-1])
cart.head()

In [27]:
# Merge the Cart data if the timestamp of user's action lies between the beginning timestamp and the (ending timestamp + 60 minutes) of the session

# It is highly inefficient to compare all the timestamps of Raw data and Cart data
# Comparing only the timestamps of the same date is much more efficient, thus create "date" column in the Raw and Cart data
# But in this case, problem arises when the session's ending time lies betweeen 23:00 and 00:00, and the timestamp of the action in the Cart data lies between 00:00 and 01:00
# Therefore we create "date2(1 day substractd from the date of the Cart data)" column in the Cart data

raw['date'] = raw['datetime'].dt.floor('d')
cart['date'] = cart['event_ymdt'].dt.floor('d')
cart['date2'] = cart['event_ymdt'].dt.floor('d') - timedelta(days=1)

In [ ]:
# Merge Raw and Cart data if "date" column coincides
# Remove data if the timestamp of the user's action does not lie between the beginning timestamp and the (ending timestamp + 60 minutes) of the session

log_cart = raw.merge(cart,on=['user_id','date'],how='inner')
log_cart = log_cart[((log_cart['event_ymdt'] - log_cart['begin_time'])>timedelta(0))*((log_cart['end_time2']-log_cart['event_ymdt'])>timedelta(0))==1]
log_cart.sort_values(by=['id','datetime'],inplace=True)

In [ ]:
# Merge Raw and Cart data if "date" column of Raw data and "date2(1 day substractd from the date of the Cart data)" column of Cart data coincides
# Remove data if the timestamp of the user's action does not lie between the beginning timestamp and the (ending timestamp + 60 minutes) of the session

log_cart2 = raw.merge(cart,left_on=['user_id','date'],right_on=['user_id','date2'],how='inner')
log_cart2 = log_cart2[((log_cart2['event_ymdt'] - log_cart2['begin_time'])>timedelta(0))*((log_cart2['end_time2']-log_cart2['event_ymdt'])>timedelta(0))==1]
log_cart2.sort_values(by=['id','datetime'],inplace=True)

In [30]:
# Concatenate above two data
log_cart = pd.concat([log_cart,log_cart2],axis=0)

# Match the action in the Cart data to the nearest (in time) log within the session
log_cart['diff'] = ((log_cart['event_ymdt'] - log_cart['datetime']).astype('timedelta64[s]'))**2
log_cart = log_cart.merge(pd.DataFrame(log_cart.groupby('id')['diff'].min()).rename(columns={'diff':'min_diff'}),on='id',how='left')
log_cart = log_cart[(log_cart['diff']-log_cart['min_diff'])**2<0.1]
log_cart.drop_duplicates(subset=['id','event_ymdt'],keep='last',inplace=True)
log_cart.drop('min_diff',axis=1,inplace=True)
log_cart['cart_id'] = 1

In [31]:
# Only 30% of the Cart data are matched to the log data 
# Unmatched data are estimated to be the actions on the web environment (we only used the log data from the mobile environment)
log_cart.shape[0] / cart.shape[0]

0.2997509047328101

In [32]:
# Drop the columns that were created but no longer useful
# Add the suffix "cart_" to the columns created in this section, to avoid the ambiguity

log_cart.drop(['cart_id','end_time2','date','date_x','date_y','date2'],axis=1,inplace=True)
log_cart.rename(columns={'prod_no':'cart_prod_no','prod_nm':'cart_prod_nm','event_type':'cart_event_type','event_ymdt':'cart_event_ymdt',
                                    'catg_nm_l':'cart_catg_nm_l','catg_nm_m':'cart_catg_nm_m','catg_nm_s':'cart_catg_nm_s','diff':'log_cart_time_diff'},inplace=True)

## 2.2. Merge with the Like data

> If there exists an action of the particular user in Like.csv, check if any of that user's session (defined by the log data) contains the action's timestamp



> If such session session exists, match the action in Like.csv to the nearest (in time) log in that session

In [ ]:
# Sort the Cart data by user_id and timestamp
# Since Like data is recorded based on UTC system, we do not need to substract 9 hours
# Drop the useless column (dt)

like.sort_values(by=['user_id','keep_reg_ymdt'],inplace=True)
like['keep_reg_ymdt'] = pd.to_datetime(like['keep_reg_ymdt'])
like.drop('dt',axis=1,inplace=True)
like.head()

In [36]:
# Merge the Like data if the timestamp of user's action lies between the beginning timestamp and the (ending timestamp + 60 minutes) of the session

# It is highly inefficient to compare all the timestamps of Raw data and Like data
# Comparing only the timestamps of the same date is much more efficient, thus create "date" column in the Raw and Like data
# But in this case, problem arises when the session's ending time lies betweeen 23:00 and 00:00, and the timestamp of the action in the Like data lies between 00:00 and 01:00
# Therefore we create "date2(1 day substractd from the date of the Like data)" column in the Like data

raw['date'] = raw['datetime'].dt.floor('d')
like['date'] = like['keep_reg_ymdt'].dt.floor('d')
like['date2'] = like['keep_reg_ymdt'].dt.floor('d') - timedelta(days=1)

In [ ]:
# Merge Raw and Like data if "date" column coincides
# Remove data if the timestamp of the user's action does not lie between the beginning timestamp and the (ending timestamp + 60 minutes) of the session

log_like = raw.merge(like,on=['user_id','date'],how='inner')
log_like = log_like[((log_like['keep_reg_ymdt'] - log_like['begin_time'])>timedelta(0))*((log_like['end_time2']-log_like['keep_reg_ymdt'])>timedelta(0))==1]
log_like.sort_values(by=['id','datetime'],inplace=True)

In [ ]:
# Merge Raw and Like data if "date" column of Raw data and "date2(1 day substractd from the date of the Cart data)" column of Like data coincides
# Remove data if the timestamp of the user's action does not lie between the beginning timestamp and the (ending timestamp + 60 minutes) of the session

log_like2 = raw.merge(like,left_on=['user_id','date'],right_on=['user_id','date2'],how='inner')
log_like2 = log_like2[((log_like2['keep_reg_ymdt'] - log_like2['begin_time'])>timedelta(0))*((log_like2['end_time2']-log_like2['keep_reg_ymdt'])>timedelta(0))==1]
log_like2.sort_values(by=['id','datetime'],inplace=True)

In [39]:
# Concatenate above two data
log_like = pd.concat([log_like,log_like2],axis=0)

# Match the action in the Like data to the nearest (in time) log within the session
log_like['diff'] = ((log_like['keep_reg_ymdt'] - log_like['datetime']).astype('timedelta64[s]'))**2
log_like = log_like.merge(pd.DataFrame(log_like.groupby('id')['diff'].min()).rename(columns={'diff':'min_diff'}),on='id',how='left')
log_like = log_like[(log_like['diff']-log_like['min_diff'])**2<0.1]
log_like.drop_duplicates(subset=['id','keep_reg_ymdt'],keep='last',inplace=True)
log_like.drop('min_diff',axis=1,inplace=True)
log_like['like_id'] = 1

In [40]:
# Only 24% of the Like data are matched to the log data 
# Unmatched data are estimated to be the actions on the web environment (we only used the log data from the mobile environment)

log_like.shape[0] / like.shape[0]

0.2465842621029442

In [41]:
# Drop the columns that were created but no longer useful
# Add the suffix "like_" to the columns created in this section, to avoid the ambiguity

log_like.drop(['like_id','end_time2','date','date2','date_x','date_y'],axis=1,inplace=True)
log_like.rename(columns={'keep_stat_cd':'like_keep_stat_cd','prod_no':'like_prod_no','prod_nm':'like_prod_nm','keep_reg_ymdt':'like_keep_reg_ymdt',
                                    'catg_nm_l':'like_catg_nm_l','catg_nm_m':'like_catg_nm_m','catg_nm_s':'like_catg_nm_s','diff':'log_like_time_diff'},inplace=True)

## 2.3. Merge with the Purchase data

> If there exists an action of the particular user in Purchase.csv, check if any of that user's session (defined by the log data) contains the action's timestamp



> If such session session exists, match the action in Purchase.csv to the nearest (in time) log in that session

In [ ]:
# Sort the Purchase data by user_id and timestamp
# Since Purchase data is recorded based on UTC system, we do not need to substract 9 hours

purchase.sort_values(by=['user_id','order_ymdt'],inplace=True)
purchase['order_ymdt'] = pd.to_datetime(purchase['order_ymdt'].apply(lambda x: '20'+x))
purchase.head()

In [45]:
# Merge the Purchase data if the timestamp of user's action lies between the beginning timestamp and the (ending timestamp + 60 minutes) of the session

# It is highly inefficient to compare all the timestamps of Raw data and Purchase data
# Comparing only the timestamps of the same date is much more efficient, thus create "date" column in the Raw and Purchase data
# But in this case, problem arises when the session's ending time lies betweeen 23:00 and 00:00, and the timestamp of the action in the Purchase data lies between 00:00 and 01:00
# Therefore we create "date2(1 day substractd from the date of the Purchase data)" column in the Purchase data 

raw['date'] = raw['datetime'].dt.floor('d')
purchase['date'] = purchase['order_ymdt'].dt.floor('d')
purchase['date2'] = purchase['order_ymdt'].dt.floor('d') - timedelta(days=1)

In [ ]:
# Merge Raw and Purchase data if "date" column coincides
# Remove data if the timestamp of the user's action does not lie between the beginning timestamp and the (ending timestamp + 60 minutes) of the session

log_purchase = raw.merge(purchase,on=['user_id','date'],how='inner')
log_purchase = log_purchase[((log_purchase['order_ymdt'] - log_purchase['begin_time'])>timedelta(0))*((log_purchase['end_time2']-log_purchase['order_ymdt'])>timedelta(0))==1]
log_purchase.sort_values(by=['id','datetime'],inplace=True)

In [ ]:
# Merge Raw and Purchase data if "date" column of Raw data and "date2(1 day substractd from the date of the Cart data)" column of Purchase data coincides
# Remove data if the timestamp of the user's action does not lie between the beginning timestamp and the (ending timestamp + 60 minutes) of the session

log_purchase2 = raw.merge(purchase,left_on=['user_id','date'],right_on=['user_id','date2'],how='inner')
log_purchase2 = log_purchase2[((log_purchase2['order_ymdt'] - log_purchase2['begin_time'])>timedelta(0))*((log_purchase2['end_time2']-log_purchase2['order_ymdt'])>timedelta(0))==1]
log_purchase2.sort_values(by=['id','datetime'],inplace=True)

In [48]:
# Concatenate above two data
log_purchase = pd.concat([log_purchase,log_purchase2],axis=0)

# Match the action in the Purchase data to the nearest (in time) log within the session
log_purchase['diff'] = ((log_purchase['order_ymdt'] - log_purchase['datetime']).astype('timedelta64[s]'))**2
log_purchase = log_purchase.merge(pd.DataFrame(log_purchase.groupby('id')['diff'].min()).rename(columns={'diff':'min_diff'}),on='id',how='left')
log_purchase = log_purchase[(log_purchase['diff']-log_purchase['min_diff'])**2<0.1]
log_purchase.drop_duplicates(subset=['id','order_ymdt'],keep='last',inplace=True)
log_purchase.drop('min_diff',axis=1,inplace=True)
log_purchase['purchase_id'] = 1

In [49]:
# Only 34% of the Purchase data are matched to the log data 
# Unmatched data are estimated to be the actions on the web environment (we only used the log data from the mobile environment)

log_purchase.shape[0] / purchase.shape[0]

0.34646126547566236

In [50]:
# Drop the columns that were created but no longer useful
# Add the suffix "purchase_" to the columns created in this section, to avoid the ambiguity

log_purchase.drop(['purchase_id','end_time2','date','date2','date_x','date_y'],axis=1,inplace=True)
log_purchase.rename(columns={'prod_order_no':'purchase_prod_order_no','order_no':'purchase_order_no','order_ymdt':'purchase_order_ymdt',
                                            'prod_no':'purchase_prod_no','prod_nm':'purchase_prod_nm','gmv':'purchase_gmv','flag':'purchase_flag',
                                            'catg_nm_l':'purchase_catg_nm_l','catg_nm_m':'purchase_catg_nm_m','catg_nm_s':'purchase_catg_nm_s','diff':'log_purchase_time_diff'},inplace=True)

## 2.4. Merge the entire Raw, Cart, Like, Purchase data

In [ ]:
# Merge log_cart, log_like, log_purchase to Raw
# In the precess of merging, drop the columns that the table has in common with the Raw data
# (Since we are merging on the "id" and "session_nth" column, we do not drop these columns even if they are common in two tables)

raw_all = raw.merge(log_cart.drop(list((set(log_cart.columns).intersection(set(raw.columns))).difference(set(['id','session_nth']))),axis=1),on=['id','session_nth'],how='left')
raw_all = raw_all.merge(log_like.drop(list((set(log_like.columns).intersection(set(raw.columns))).difference(set(['id','session_nth']))),axis=1),on=['id','session_nth'],how='left')
raw_all = raw_all.merge(log_purchase.drop(list((set(log_purchase.columns).intersection(set(raw.columns))).difference(set(['id','session_nth']))),axis=1),on=['id','session_nth'],how='left')
raw_all.head(1)

In [53]:
# Create "cart_id(1 if there exists at least one Cart action within the session, otherwise 0)" column
# Create "like_id(1 if there exists at least one Like action within the session, otherwise 0)" column
# Create "purchase_id(1 if there exists at least one Purchase action within the session, otherwise 0)" column

raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['cart_event_ymdt'].count().gt(0).astype(float)).rename(columns={'cart_event_ymdt':'cart_id'}),on='id',how='left')
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['like_keep_reg_ymdt'].count().gt(0).astype(float)).rename(columns={'like_keep_reg_ymdt':'like_id'}),on='id',how='left')
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['purchase_order_ymdt'].count().gt(0).astype(float)).rename(columns={'purchase_order_ymdt':'purchase_id'}),on='id',how='left')

In [54]:
# Drop the useless columns from raw_all

raw_all.drop(['end_time2','date'],axis=1,inplace=True)

In [55]:
raw_all.columns

Index(['datetime', 'user_id', 'sex', 'age', 'url', 'device', 'page',
       'Labeling', 'session', 'id', 'page_stay_time', 'session_length',
       'session_nth', 'prev_Labeling', 'prev_page', 'begin_time', 'end_time',
       'cart_prod_no', 'cart_prod_nm', 'cart_event_type', 'cart_event_ymdt',
       'cart_catg_nm_l', 'cart_catg_nm_m', 'cart_catg_nm_s',
       'log_cart_time_diff', 'like_keep_stat_cd', 'like_prod_no',
       'like_prod_nm', 'like_keep_reg_ymdt', 'like_catg_nm_l',
       'like_catg_nm_m', 'like_catg_nm_s', 'log_like_time_diff',
       'purchase_prod_order_no', 'purchase_order_no', 'purchase_order_ymdt',
       'purchase_prod_no', 'purchase_prod_nm', 'purchase_gmv', 'purchase_flag',
       'purchase_catg_nm_l', 'purchase_catg_nm_m', 'purchase_catg_nm_s',
       'log_purchase_time_diff', 'cart_id', 'like_id', 'purchase_id'],
      dtype='object')

# 3. Remove outlier sessions

In [56]:
# Remove the sessions with length less than 2
raw_all = raw_all[raw_all['session_length']>2]

In [57]:
raw_all.shape

(5003128, 47)

In [58]:
# Create the "distinct_page_num(number of distinct pages within the session)" column
# Create the "distinct_label_num(number of distinct labels within the session)" column
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['page'].nunique()).rename(columns={'page':'distinct_page_num'}),on='id',how='left')
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['Labeling'].nunique()).rename(columns={'Labeling':'distinct_label_num'}),on='id',how='left')

In [59]:
# Remove the sessions that consists of identical pages
raw_all = raw_all[raw_all['distinct_page_num']>1]

In [60]:
raw_all.shape

(4946055, 49)

# 4. Create features that describe each session

## 4.1. Features related to the staying time within each log/session

In [61]:
# Create "max_page_stay_time(maximum staying time of the single log within the session)" column
# Create "min_page_stay_time(minimum staying time of the single log within the session)" column

raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['page_stay_time'].max()).rename(columns={'page_stay_time':'max_page_stay_time'}),on='id',how='left')
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['page_stay_time'].min()).rename(columns={'page_stay_time':'min_page_stay_time'}),on='id',how='left')

In [62]:
# Create "mean_page_stay_time(mean of the staying time for each log within the session)" column
# If we simply apply the ".groupby.mean()", then the denominators of the mean calculations become inaccurate (substracted by 1)
# Therefore we apply ".groupby.count()+1" instead of ".groupby.sum()"

raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['page_stay_time'].sum() / (raw_all.groupby('id')['page_stay_time'].count()+1)).rename(columns={'page_stay_time':'mean_page_stay_time'}),on='id',how='left')

In [63]:
# Create the "session_stay_time(total staying time within the session)" column

raw_all['session_stay_time'] = raw_all['end_time'] - raw_all['begin_time']

In [64]:
# Data types of stay_time columns created above are "timedelta" 
# For convenience, convert the data types of these columns into "float" of seconds

raw_all['page_stay_time'] = raw_all['page_stay_time'].astype('timedelta64[s]')
raw_all['max_page_stay_time'] = raw_all['max_page_stay_time'].astype('timedelta64[s]')
raw_all['min_page_stay_time'] = raw_all['min_page_stay_time'].astype('timedelta64[s]')
raw_all['mean_page_stay_time'] = raw_all['mean_page_stay_time'].astype('timedelta64[s]')
raw_all['session_stay_time'] = raw_all['session_stay_time'].astype('timedelta64[s]')

## 4.2. Time elapsed from the beginning time of the session

In [65]:
# Create the "time_from_begin(how much time elapsed from the beginning time of the session when the corresponding log occured)" column

raw_all['time_from_begin'] = raw_all['datetime'] - raw_all['begin_time']

In [66]:
# Create "purchase_page_reach_time(If an user visited the page of Label 21[payment page], how long did it take from the beginning of the session)" column

time_from_begin = raw_all.drop_duplicates(subset=['id','Labeling'])[['id','Labeling','time_from_begin']].rename(columns={'time_from_begin':'purchase_page_reach_time'})
raw_all = raw_all.merge(time_from_begin[time_from_begin['Labeling']==21][['id','purchase_page_reach_time']],on='id',how='left')

In [67]:
# Data types of above two columns created above are "timedelta" 
# For convenience, convert the data types of these columns into "float" of seconds

raw_all['time_from_begin'] = raw_all['time_from_begin'].astype('timedelta64[s]')
raw_all['purchase_page_reach_time'] = raw_all['purchase_page_reach_time'].astype('timedelta64[s]')

## 4.3. Number of recursion (repetition number of particular label)

In [68]:
# Create "recursion_4(repetition number of Label 4)" column

recursion_4 = raw_all.groupby(['id','Labeling'])['prev_Labeling'].value_counts().unstack()[4].unstack()[4]
raw_all = raw_all.merge(pd.DataFrame(recursion_4).rename(columns={4:'recursion_4'}),on='id',how='left')

In [69]:
# Create "recursion_4(repetition number of Label 5)" column

recursion_5 = raw_all.groupby(['id','Labeling'])['prev_Labeling'].value_counts().unstack()[5].unstack()[5]
raw_all = raw_all.merge(pd.DataFrame(recursion_5).rename(columns={5:'recursion_5'}),on='id',how='left')

In [70]:
# Create "recursion_4(repetition number of Label 6)" column

recursion_6 = raw_all.groupby(['id','Labeling'])['prev_Labeling'].value_counts().unstack()[6].unstack()[6]
raw_all = raw_all.merge(pd.DataFrame(recursion_6).rename(columns={6:'recursion_6'}),on='id',how='left')

In [71]:
# Create "recursion_4(repetition number of Label 8)" column

recursion_8 = raw_all.groupby(['id','Labeling'])['prev_Labeling'].value_counts().unstack()[8].unstack()[8]
raw_all = raw_all.merge(pd.DataFrame(recursion_8).rename(columns={8:'recursion_8'}),on='id',how='left')

## 4.4. Number of Cart / Like / Purchase actions

In [72]:
# Create "cart_add_num(Number of Cart actions within the session)" column
# Create "like_add_num(Number of Like actions within the session)" column
# Create "purchase_num(Number of Purchase actiosn within the session)" column

raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['cart_event_ymdt'].count()).rename(columns={'cart_event_ymdt':'cart_add_num'}),on='id',how='left')
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['like_keep_reg_ymdt'].count()).rename(columns={'like_keep_reg_ymdt':'like_add_num'}),on='id',how='left')
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['purchase_order_ymdt'].count()).rename(columns={'purchase_order_ymdt':'purchase_num'}),on='id',how='left')

## 4.5. Features related to the particular labels within the session

In [73]:
# Create "session_begin_label(beginning label of the session)" column

raw_all = raw_all.merge(raw_all.drop_duplicates(subset='id',keep='first')[['id','Labeling']].rename(columns={'Labeling':'session_begin_label'}),on='id',how='left')

In [74]:
# Create "session_end_label(ending label of the session)" column

raw_all = raw_all.merge(raw_all.drop_duplicates(subset='id',keep='last')[['id','Labeling']].rename(columns={'Labeling':'session_end_label'}),on='id',how='left')

In [75]:
# Create "review_page_num(how many Label 7[review page] exists within the session)" columns

review_page_num = pd.DataFrame(raw_all.groupby('id')['Labeling'].value_counts().unstack()[7]).rename(columns={7:'review_page_num'})
raw_all = raw_all.merge(review_page_num,on='id',how='left')

In [76]:
raw_all.shape

(4946055, 65)

In [77]:
raw_all.columns

Index(['datetime', 'user_id', 'sex', 'age', 'url', 'device', 'page',
       'Labeling', 'session', 'id', 'page_stay_time', 'session_length',
       'session_nth', 'prev_Labeling', 'prev_page', 'begin_time', 'end_time',
       'cart_prod_no', 'cart_prod_nm', 'cart_event_type', 'cart_event_ymdt',
       'cart_catg_nm_l', 'cart_catg_nm_m', 'cart_catg_nm_s',
       'log_cart_time_diff', 'like_keep_stat_cd', 'like_prod_no',
       'like_prod_nm', 'like_keep_reg_ymdt', 'like_catg_nm_l',
       'like_catg_nm_m', 'like_catg_nm_s', 'log_like_time_diff',
       'purchase_prod_order_no', 'purchase_order_no', 'purchase_order_ymdt',
       'purchase_prod_no', 'purchase_prod_nm', 'purchase_gmv', 'purchase_flag',
       'purchase_catg_nm_l', 'purchase_catg_nm_m', 'purchase_catg_nm_s',
       'log_purchase_time_diff', 'cart_id', 'like_id', 'purchase_id',
       'distinct_page_num', 'distinct_label_num', 'max_page_stay_time',
       'min_page_stay_time', 'mean_page_stay_time', 'session_stay_time',
    

# 5. Classifing the inflow type into Search/Exploration

## 5.1. Search inflow

In [78]:
# Create "is_query_page(if thw word 'query' exists within the URL)" column
raw_all['is_query_page'] = raw_all['url'].apply(lambda x: 'query' in x)

In [ ]:
# Condition to be classified as Search inflow: query_page appears within 30 seconds after the beginning of the session & 2nd log within the session
# Create "under_30sec(if the log is within 30 seconds from the beginning of the session)" column
# Create "under_2th(if the log is 1st or 2nd log of the session)" column
# Create "search_in_log(if the log is within 30 seconds from the beginning of the session, and is 1st or 2nd log of the session & and it is query_page)" column

raw_all['under_30sec'] = pd.to_timedelta(raw_all['time_from_begin'],unit='s')<=pd.Timedelta(seconds=30)
raw_all['under_2th'] = raw_all['session_nth']<=2
raw_all['search_in_log'] = raw_all['under_30sec'] * raw_all['under_2th'] * raw_all['is_query_page']

In [80]:
# Create "search_in(if the session is search inflow session)" column

raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['search_in_log'].sum().gt(0)).astype(float).rename(columns={'search_in_log':'search_in'}),on='id',how='left')

In [81]:
# Drop useless columns created above

raw_all.drop(['under_30sec','under_2th','search_in_log'],axis=1,inplace=True)

In [82]:
raw_all.shape

(4946055, 67)

In [83]:
raw_all.drop_duplicates('id')['search_in'].value_counts()

0.0    252550
1.0     99357
Name: search_in, dtype: int64

## 5.2. Exploration inflow

In [84]:
# Condition to be classified as the Exploration inflow: Except for the Search inflow session, all the other sessions that do not consists solely of transaction(payment)/shipment logs
# Create "is_trans_revisit(whether the log is about transaction or shipment)" column

trans_revisit = [15, 21, 22, 23,17, 18, 19, 20, 25, 26]
raw_all['is_trans_revisit'] = raw_all['Labeling'].apply(lambda x: x in trans_revisit)

In [85]:
# Create "trans_revisit_sum(number of trans_revisit logs within the session)" column

raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['is_trans_revisit'].sum()).rename(columns={'is_trans_revisit':'trans_revisit_sum'}),on='id',how='left')

In [86]:
# Create trans_revisit(whether the session consists solely of transaction(payment)/shipment logs)" column
# When calculating this column, we exclude the Search sessions that consists soly of transaction(payment)/shipment logs

raw_all['trans_revisit'] = (raw_all['trans_revisit_sum'] == raw_all['session_length']).astype(float) * (1-raw_all['search_in'])

In [87]:
raw_all.drop_duplicates('id').groupby('trans_revisit')['search_in'].value_counts().unstack()

search_in,0.0,1.0
trans_revisit,,
0.0,228547.0,99357.0
1.0,24003.0,NaN


In [88]:
# Drop useless columns created above

raw_all.drop(['is_trans_revisit','trans_revisit_sum'],axis=1,inplace=True)

In [89]:
# Create "in_type(to which inflow category the session belongs)" column

def compute_in_type(x):
    if x==2:
        return '검색유입'
    elif x==1:
        return '결제/배송'
    elif x==0:
        return '탐색유입'
    else:
        return np.nan

raw_all['in_type'] = (raw_all['search_in']*2 + raw_all['trans_revisit']).apply(lambda x: compute_in_type(x))

In [90]:
pd.DataFrame(raw_all.drop_duplicates('id')['in_type'].value_counts())

,in_type
탐색유입,228547
검색유입,99357
결제/배송,24003


In [91]:
# Drop useless columns created above

raw_all.drop(['search_in','trans_revisit'],axis=1,inplace=True)

In [92]:
raw_all.shape

(4946055, 67)

In [93]:
raw_all.columns

Index(['datetime', 'user_id', 'sex', 'age', 'url', 'device', 'page',
       'Labeling', 'session', 'id', 'page_stay_time', 'session_length',
       'session_nth', 'prev_Labeling', 'prev_page', 'begin_time', 'end_time',
       'cart_prod_no', 'cart_prod_nm', 'cart_event_type', 'cart_event_ymdt',
       'cart_catg_nm_l', 'cart_catg_nm_m', 'cart_catg_nm_s',
       'log_cart_time_diff', 'like_keep_stat_cd', 'like_prod_no',
       'like_prod_nm', 'like_keep_reg_ymdt', 'like_catg_nm_l',
       'like_catg_nm_m', 'like_catg_nm_s', 'log_like_time_diff',
       'purchase_prod_order_no', 'purchase_order_no', 'purchase_order_ymdt',
       'purchase_prod_no', 'purchase_prod_nm', 'purchase_gmv', 'purchase_flag',
       'purchase_catg_nm_l', 'purchase_catg_nm_m', 'purchase_catg_nm_s',
       'log_purchase_time_diff', 'cart_id', 'like_id', 'purchase_id',
       'distinct_page_num', 'distinct_label_num', 'max_page_stay_time',
       'min_page_stay_time', 'mean_page_stay_time', 'session_stay_time',
    

# 6. Save as csv file & Save as pickle file after the memory optimization

In [94]:
# Save the entire data into the csv file
raw_all.to_csv(os.path.join(os.getcwd(),'final_log.csv'),index=False)

In [8]:
# Cited from https://data-newbie.tistory.com/472

def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [96]:
mem_usage(raw_all)

'9760.81 MB'

In [9]:
# Cited from https://data-newbie.tistory.com/472

def int_memory_reduce(data) :
    data_int = data.select_dtypes(include=['int'])
    converted_int = data_int.apply(pd.to_numeric,downcast='unsigned')
    print(f"Before : {mem_usage(data_int)} -> After : {mem_usage(converted_int)}")
    data[converted_int.columns] = converted_int
    return data

## 연속형 데이터 사이즈 축소 함소
def float_memory_reduce(data) :
    data_float = data.select_dtypes(include=['float'])
    converted_float = data_float.apply(pd.to_numeric,downcast='float')
    print(f"Before : {mem_usage(data_float)} -> After : {mem_usage(converted_float)}")
    data[converted_float.columns] = converted_float
    return data

## 문자형 데이터 사이즈 축소 함소
def object_memory_reduce(data) :
    gl_obj = data.select_dtypes(include=['object']).copy()
    converted_obj = pd.DataFrame()
    for col in gl_obj.columns:
        num_unique_values = len(gl_obj[col].unique())
        num_total_values = len(gl_obj[col])
        if num_unique_values / num_total_values < 0.5:
            converted_obj.loc[:,col] = gl_obj[col].astype('category')
        else:
            converted_obj.loc[:,col] = gl_obj[col]
    print(f"Before : {mem_usage(gl_obj)} -> After : {mem_usage(converted_obj)}")
    data[converted_obj.columns] = converted_obj
    return data

In [98]:
raw_all2 = raw_all.copy()
raw_all2 = int_memory_reduce(raw_all2)
raw_all2 = float_memory_reduce(raw_all2)
raw_all2 = object_memory_reduce(raw_all2)

/tmp/ipykernel_5209/1932203267.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  usage_b = pandas_obj.memory_usage(deep=True).sum()
/tmp/ipykernel_5209/1932203267.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  usage_b = pandas_obj.memory_usage(deep=True).sum()


Before : 37.74 MB -> After : 37.74 MB
Before : 943.39 MB -> After : 490.56 MB
Before : 8232.66 MB -> After : 1256.21 MB


In [99]:
mem_usage(raw_all2)

'2392.99 MB'

In [100]:
raw_all2.to_pickle(os.path.join(os.getcwd(),'final_log.pkl'))

# 7. Leave only the sessions with length less than 50

In [3]:
raw_all = pd.read_csv(os.path.join(os.getcwd(),'final_log.csv'))

/tmp/ipykernel_2948/2580486793.py:1: DtypeWarning: Columns (25,27,28,29,30,31,36) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_all = pd.read_csv(os.path.join(os.getcwd(),'final_log(cart buy zzim).csv'))


In [5]:
raw_all_under50 = raw_all[raw_all['session_length']<=50]

In [7]:
raw_all_under50.to_csv(os.path.join(os.getcwd(),'final_log_under50.csv'),index=False)

In [10]:
mem_usage(raw_all_under50)

'7547.21 MB'

In [12]:
raw_all_under50_small = raw_all_under50.copy()
raw_all_under50_small = int_memory_reduce(raw_all_under50_small)
raw_all_under50_small = float_memory_reduce(raw_all_under50_small)
raw_all_under50_small = object_memory_reduce(raw_all_under50_small)

Before : 359.19 MB -> After : 72.53 MB
Before : 690.75 MB -> After : 359.19 MB
Before : 6874.22 MB -> After : 1310.79 MB


In [13]:
mem_usage(raw_all_under50_small)

'1690.70 MB'

In [14]:
raw_all_under50_small.to_pickle(os.path.join(os.getcwd(),'final_log_under50.pkl'))